In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse import csr_matrix
import time
import csv

In [3]:
df=pd.read_csv('../ml-20m/ratings.csv')

In [6]:
df.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [7]:
df.iloc[1000:1110]


,userId,movieId,rating,timestamp
1000,11,527,4.5,1251170861
1001,11,531,4.5,1230782539
1002,11,541,4.5,1230787503
1003,11,546,5.0,1230788928
1004,11,551,5.0,1230853957
1005,11,586,4.0,1230859027
1006,11,587,4.5,1230858969
1007,11,588,5.0,1251170867
1008,11,589,4.5,1230788848
1009,11,592,4.5,1230789445


In [42]:
df=df.iloc[1:1110]#In this program we omit some data according to memory we have on access.We can test it on real data on LAB by then.

In [9]:
df

,userId,movieId,rating,timestamp
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826
10,1,293,4.0,1112484703


In [10]:
matrix = df.pivot(index ='movieId', columns='userId', values='rating').fillna(0)
matrix.shape


(742, 11)

In [11]:
matrix_sparse = csr_matrix(matrix)

In [12]:
matrix_sparse.shape

(742, 11)

In [13]:
item_similarity = pairwise_distances(matrix_sparse, metric='cosine')
user_similarity = pairwise_distances(matrix_sparse.T, metric='cosine')

In [14]:
item_similarity.shape 
user_similarity.shape 

(11, 11)

In [15]:
# Make prediction
def predict(matrix, similarity, type='user'):
    if type == 'user':
        mean_user_rating = matrix.mean(axis=1)
        ratings_diff = (matrix - mean_user_rating)
        pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = matrix.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

item_prediction = predict(matrix_sparse.T, item_similarity, type='item')
user_prediction = predict(matrix_sparse.T, user_similarity, type='user')

In [17]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic, evaluate

data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], Reader(rating_scale=(1, df['rating'].max())))

# First, train the algortihm to compute the similarities between items
# training is very slow..
trainset = data.build_full_trainset()

# compute cosine similarities between items
sim_options = {'name': 'cosine','user_based': False}
knn = KNNBasic(k=5, sim_options=sim_options)
knn.fit(trainset)



Computing the cosine similarity matrix...
Done computing similarity matrix.


In [36]:
# predict a certain item
userid = 11
itemid = 2428
print(knn.predict(userid, itemid))

user: 11         item: 2428       r_ui = None   est = 3.50   {'actual_k': 5, 'was_impossible': False}


In [41]:
# actual rating #error is for data is not complete.
print("Actual no by user 0 =", matrix.loc[2428][3])

Actual no by user 0 = 4.0
